In [1]:
%pip install opik openai comet_ml litellm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.7/304.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.1 MB/s eta 0:00:00


In [2]:
import opik
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import (Hallucination, AnswerRelevance)
from opik.integrations.openai import track_openai
import openai
import os
from datetime import datetime
from getpass import getpass
import litellm
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

# Define project name to enable tracing
os.environ["OPIK_PROJECT_NAME"] = "llm-based-eval"


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "bluemusk" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [4]:
# Hugging Face Configs to access meta-llama-3.2 model
if "HF_TOKEN" not in os.environ:
  os.environ["HF_TOKEN"] = getpass("Enter your Hugging Face Key: ")

Enter your Hugging Face Key: ··········


In [5]:
# access Opik client and llama3.2
client = Opik()

MODEL = "huggingface/meta-llama/Llama-3.2-3B-Instruct"

# Prompts & Templates

In [6]:
prompt_template = """Use the following context to answer my question:

### CONTEXT:
{context}

### QUESTION:
{question}
"""

# Dataset

In [7]:
# Create dataset
dataset = client.get_or_create_dataset(
    name="HaluBench", description="HaluBench dataset"
)

In [8]:
import pandas as pd

df = pd.read_parquet(
    "hf://datasets/PatronusAI/HaluBench/data/test-00000-of-00001.parquet"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
df.head()

,id,passage,question,answer,label,source_ds
0,d3fb4c3c-d21b-480a-baa0-98d6d0d17c1d,Hoping to rebound from the road loss to the Ch...,Which team scored the longest field goal kick ...,"['Rams', 'second', 'Marc Bulger', 'Kevin Curtis']",FAIL,DROP
1,8603663e-c53b-46db-a482-a867f12ff3b4,"As of the census of 2000, there were 218,590 p...",How many percent were not Irish?,87.1,FAIL,DROP
2,c63a73e5-2c91-489b-bd24-af150ddfa82c,Hoping to rebound from the road loss to the Ch...,How many yards was the second longest field go...,42,FAIL,DROP
3,52db14ed-5426-46ec-b0ae-4ef843b2d692,Hoping to rebound from their tough overtime ro...,How long was the last touchdown?,18-yard,FAIL,DROP
4,31b36417-aad1-412c-b0e5-9c1faaed233f,"As of the census of 2000, there were 218,590 p...",How many in percent from the census weren't Ir...,87.1,FAIL,DROP


In [10]:
cleaned_ds = df.drop(['answer', 'label', 'source_ds', 'id'], axis=1).iloc[0:100]
cleaned_ds.head()

,passage,question
0,Hoping to rebound from the road loss to the Ch...,Which team scored the longest field goal kick ...
1,"As of the census of 2000, there were 218,590 p...",How many percent were not Irish?
2,Hoping to rebound from the road loss to the Ch...,How many yards was the second longest field go...
3,Hoping to rebound from their tough overtime ro...,How long was the last touchdown?
4,"As of the census of 2000, there were 218,590 p...",How many in percent from the census weren't Ir...


In [11]:
dataset.insert(cleaned_ds.to_dict('records'))
# read the above dataset
dataset.to_pandas().head()

,passage,question,id
0,"Trying to snap a two-game skid, the Bills flew...",How many games had the Bills won before this g...,01944c76-76ee-7393-bf6e-0f3dcc1309cd
1,1564: The city of Ryazan posad was burned.:47 ...,What was burned first: city of Ryazan or subur...,01944c76-76ed-7652-a0b8-fa31ad153fc4
2,"As of the census of 2000, there were 218,590 p...",How many percent were not Italian?,01944c76-76ec-7c91-99af-f3203c42d819
3,"As of the census of 2000, there were 218,590 p...",Which group from the census is smaller: German...,01944c76-76eb-7082-a125-5bc156c92cd5
4,"In week 6, the Lions hosted the NFC West Divis...",How many field goals between 20 and 30 yards w...,01944c76-76ea-7b65-a465-ef851a718daf


# LLM Application with Litellm

In [12]:
# function call of llama3 using litellm
@track
def chatbot_application(question: str, context: str) -> str:
    response = litellm.completion(
        model=MODEL,
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user", "content":prompt_template.format(context=context, question=question)}
        ]
    )
    return response.choices[0].message.content

# Evaluation

In [13]:
# Define the evaluation task
def evaluation_task(x):
    return {
        "input": x['question'],
        "output": chatbot_application(x['question'], x['passage']),
        "context": x['passage']
    }

In [14]:
# Retrieve the dataset
client = Opik()

In [15]:
# Define the metrics with model meta-llama/Llama-3.2.1B-Instruct
metrics = [AnswerRelevance(model=MODEL)]

# experiment_name
experiment_name = MODEL + "_" + dataset.name + "_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# run evaluation
evaluation = evaluate(
    experiment_name=experiment_name,
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=metrics,
    experiment_config={
        "model": MODEL
    }
)

Evaluation:   0%|          | 0/96 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/opik/evaluation/metrics/llm_judges/answer_relevance/metric.py", line 123, in _parse_model_output
    dict_content = json.loads(content)
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (cha

╭─ HaluBench (96 samples) ──────────────────────────╮
│                                                   │
│ Total time:        00:00:37                       │
│ Number of samples: 96                             │
│                                                   │
│ answer_relevance_metric: 0.6222 (avg) - 87 failed │
│                                                   │
╰───────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=985852;https://www.comet.com/opik/bluemusk/experiments/01944c75-04b5-75a5-9b7a-b01456ac795a/compare?experiments=%5B%2201944dff-be1c-7ca2-a18d-1ca6afa035d0%22%5D\in your Opik dashboard]8;;\.